In [0]:
dbutils.widgets.text("indegree_parquet", "/FileStore/tables/cfuentes/indegrees_test.parquet")
dbutils.widgets.text("english_documents", "/FileStore/tables/cfuentes/english_articles.parquet")

In [0]:
from pyspark.sql.functions import explode

parquet_path = dbutils.widgets.get("english_documents")
parquet_inDegrees = dbutils.widgets.get("indegree_parquet")

parquetDF = spark.read.parquet(parquet_path)



In [0]:
from graphframes import *

vertex = parquetDF.select(explode("authors").alias("id")).distinct()

edges = parquetDF.select("paper_id", explode("authors").alias("r_author")).alias("left").join(parquetDF.select("paper_id", explode("authors").alias("l_author")).alias("right"), on=["paper_id"]) \
.select("l_author", "r_author").filter("'l_author' != 'r_author'").distinct().withColumnRenamed("l_author", "src").withColumnRenamed("r_author", "dst")

g = GraphFrame(vertex, edges)

In [0]:
g.inDegrees.write.mode("append").parquet(parquet_inDegrees)

In [0]:
from pyspark.sql.functions import when, col

processedDF = parquetDF.select("paper_id", explode("authors")).select(col("paper_id").alias("id")).distinct()

left_join = parquetDF.join(processedDF, processedDF.id == parquetDF.paper_id, how='left') 

left_join = left_join.withColumn("graph_has_errors", when(col("id").isNotNull(), False).otherwise(True))

col_list = parquetDF.columns + ["graph_has_errors"]

left_join.select(*col_list).write.mode('overwrite').parquet(parquet_path)

In [0]:
dbutils.notebook.exit("Ok")